<a href="https://colab.research.google.com/github/Serena-G-LEE/25-2_Project/blob/main/AI_cross_department_relationship_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 셀 1: 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 셀 2: 라이브러리 임포트 및 기본 경로 설정
import os
import glob
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 선생님 구글 드라이브 내 프로젝트 폴더 (필요하면 여기만 수정하시면 됩니다)
BASE_DIR = "/content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트"

# 연도별 폴더 이름 매핑
# 2011년 예시는 주어진 그대로 사용했고,
# 나머지 연도는 실제 폴더 이름에 맞게 선생님께서 직접 채워 넣으시면 됩니다.
YEAR_FOLDER_MAP = {
    2011: "openalex_data_cleaning_2011_with_authors",
    2012: "openalex_data_cleaning_2012_with_authors",
    2013: "openalex_data_cleaning_2013_with_authors",
    2014: "openalex_data_cleaning_2014_with_authors",
    2015: "openalex_data_cleaning_2015_with_authors",
    2016: "openalex_data_cleaning_2016_with_authors",
    2017: "openalex_data_cleaning_2017_with_authors",
    2018: "openalex_data_cleaning_2018_with_authors",
    2019: "openalex_data_cleaning_2019_with_authors",
    2020: "openalex_data_cleaning_2020_with_authors",
    2021: "openalex_data_cleaning_2021_with_authors",
    2022: "openalex_data_cleaning_2022_with_authors",
    2023: "openalex_data_cleaning_2023_with_authors",
    2024: "openalex_data_cleaning_2024_with_authors",
    2025: "openalex_data_cleaning_2025_with_authors",
}

# 텍스트 결합에 사용할 컬럼
# 정제된 제목/초록 컬럼을 사용합니다.
TEXT_COLUMNS = ["title_clean", "abstract_clean"]

print("BASE_DIR:", BASE_DIR)
print("YEAR_FOLDER_MAP:", YEAR_FOLDER_MAP)
print("TEXT_COLUMNS:", TEXT_COLUMNS)

CHECKPOINT_DIR = os.path.join(BASE_DIR, "checkpoints")
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print("CHECKPOINT_DIR:", CHECKPOINT_DIR)


BASE_DIR: /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트
YEAR_FOLDER_MAP: {2011: 'openalex_data_cleaning_2011_with_authors', 2012: 'openalex_data_cleaning_2012_with_authors', 2013: 'openalex_data_cleaning_2013_with_authors', 2014: 'openalex_data_cleaning_2014_with_authors', 2015: 'openalex_data_cleaning_2015_with_authors', 2016: 'openalex_data_cleaning_2016_with_authors', 2017: 'openalex_data_cleaning_2017_with_authors', 2018: 'openalex_data_cleaning_2018_with_authors', 2019: 'openalex_data_cleaning_2019_with_authors', 2020: 'openalex_data_cleaning_2020_with_authors', 2021: 'openalex_data_cleaning_2021_with_authors', 2022: 'openalex_data_cleaning_2022_with_authors', 2023: 'openalex_data_cleaning_2023_with_authors', 2024: 'openalex_data_cleaning_2024_with_authors', 2025: 'openalex_data_cleaning_2025_with_authors'}
TEXT_COLUMNS: ['title_clean', 'abstract_clean']
CHECKPOINT_DIR: /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/checkpoints


In [ ]:
# 셀 3: 여러 연도에 대해 폴더 구조와 컬럼 확인

# 2011년부터 2026년까지 폴더와 파일 구조를 확인합니다.
years_to_check = range(2011, 2027)  # 2011, 2012, ..., 2026

# 만약 단일 연도만 보고 싶으시면 위 줄을 주석 처리하고
# 예를 들어 2015년만 보고 싶을 때는 아래처럼 바꾸시면 됩니다.
# years_to_check = [2015]

for sample_year in years_to_check:
    print("\n==============================")
    print("연도:", sample_year)

    if sample_year not in YEAR_FOLDER_MAP:
        print(f"{sample_year}년은 YEAR_FOLDER_MAP에 아직 등록되지 않았습니다.")
        continue

    year_folder = YEAR_FOLDER_MAP[sample_year]
    folder_path = os.path.join(BASE_DIR, year_folder)

    print("폴더 경로:", folder_path)
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    print(f"{sample_year}년 폴더 내 CSV 파일 수:", len(csv_files))
    for f in csv_files[:5]:
        print(" -", os.path.basename(f))

    # ALL 파일 하나를 예시로 열어서 컬럼 구조를 확인합니다.
    all_pattern = os.path.join(folder_path, f"works_top10000_{sample_year}_ALL.cleaned.csv")
    all_files = glob.glob(all_pattern)

    if len(all_files) > 0:
        print("\n예시로 읽는 파일:", all_files[0])
        df_sample = pd.read_csv(all_files[0])
        print("샘플 파일 컬럼:")
        print(df_sample.columns)
    else:
        print("works_top10000_YYYY_ALL.cleaned.csv 패턴에 맞는 파일을 찾지 못했습니다.")



연도: 2011
폴더 경로: /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/openalex_data_cleaning_2011_with_authors
2011년 폴더 내 CSV 파일 수: 25
 - institutions_mapping.cleaned.csv
 - works_top10000_2011_ALL.cleaned.csv
 - works_top10000_2011_Chonnam_National_University.cleaned.csv
 - works_top10000_2011_Chung-Ang_University.cleaned.csv
 - works_top10000_2011_Chungbuk_National_University.cleaned.csv

예시로 읽는 파일: /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/openalex_data_cleaning_2011_with_authors/works_top10000_2011_ALL.cleaned.csv
샘플 파일 컬럼:
Index(['openalex_id', 'title', 'publication_year', 'publication_date', 'type',
       'host_venue', 'cited_by_count', 'is_oa', 'oa_status', 'language', 'doi',
       'abstract', 'authors_count', 'author_ids', 'author_names',
       'author_positions', 'author_orcids', 'author_affiliations',
       'query_name', 'display_name', 'institution_id', '_garbled_score',
       'title_clean', 'host_venue_clean', 'language_clean', 'abstract_clean',
  

In [ ]:
# 셀 4: 인공지능 관련 키워드 및 세부 분야 정의 (소문자 기준으로 작성)

# AI 여부를 판단할 때 사용할 전체 키워드 목록입니다.
# 실제 매칭에서는 텍스트를 소문자로 변환하고, 정규식 검색도 대소문자 무시 옵션을 사용하므로
# 원문이 대문자든 소문자든 모두 잡힙니다.

# 인공지능 세부 분야별 키워드와 한글 이름 정의
AI_AREAS = {
    "ml_general": {
        "name_ko": "머신러닝 일반",
        "keywords": [
            # 일반 머신러닝 개념
            "machine learning",
            "statistical learning",
            "supervised learning",
            "unsupervised learning",
            "semi supervised learning",
            "semi-supervised learning",
            "self supervised learning",
            "self-supervised learning",
            "unsupervised representation learning",
            "representation learning",
            "feature learning",
            "feature extraction",
            "pattern recognition",
            "knowledge discovery",

            # 데이터 분석·예측
            "data mining",
            "text mining",
            "predictive modeling",
            "predictive model",
            "predictive analytics",
            "data analytics",
            "data analysis",
            "big data analytics",
            "data-driven model",
            "data-driven modeling",
            "statistical modeling",
            "statistical model",
            "regression model",
            "classification model",

            # 분석 기법
            "clustering analysis",
            "cluster analysis",
            "time series analysis",
            "spatio temporal analysis",
            "spatio-temporal analysis",
            "multivariate analysis",
            "dimensionality reduction",
            "feature selection",

            # 학습 전략
            "online learning",
            "incremental learning",
            "adaptive learning",
            "ensemble learning",
            "model selection",
            "model validation",

            # 최적화·전달학습
            "hyperparameter optimization",
            "hyperparameter tuning",
            "hyper-parameter optimization",
            "meta-learning",
            "transfer learning",
            "domain adaptation",
            "multi-task learning",
            "few-shot learning",
            "zero-shot learning",

            # 이상 탐지 계열
            "anomaly detection",
            "outlier detection",
            "novelty detection",
            "fault diagnosis",
            "condition monitoring",
        ],
    },
    "dl_architecture": {
        "name_ko": "신경망 및 딥러닝 구조",
        "keywords": [
            # 기본 신경망
            "neural network",
            "neural networks",
            "artificial neural network",
            "artificial neural networks",
            "deep neural network",
            "deep neural networks",
            "feedforward neural network",
            "multilayer perceptron",
            "mlp",

            # 합성곱 신경망
            "convolutional neural network",
            "convolutional neural networks",
            "cnn",
            "convnet",
            "residual network",
            "resnet",
            "vgg network",
            "inception network",
            "unet",
            "encoder decoder network",
            "encoder-decoder network",

            # 순환 신경망
            "recurrent neural network",
            "recurrent neural networks",
            "rnn",
            "long short term memory",
            "long short-term memory",
            "lstm",
            "bidirectional lstm",
            "gated recurrent unit",
            "gru",
            "sequence to sequence model",
            "sequence-to-sequence model",
            "seq2seq",

            # 트랜스포머·어텐션
            "deep learning",
            "attention mechanism",
            "self-attention",
            "multi-head attention",
            "transformer",
            "transformer-based model",
            "vision transformer",
            "vit",

            # 그래프 신경망
            "graph neural network",
            "graph neural networks",
            "gnn",
            "graph convolutional network",
            "gcn",
            "graph attention network",
            "gat",
            "graph autoencoder",

            # 생성·표현 모델
            "autoencoder",
            "autoencoders",
            "variational autoencoder",
            "variational autoencoders",
            "vae",
            "denoising autoencoder",
            "sparse autoencoder",
            "generative adversarial network",
            "generative adversarial networks",
            "gan",
            "conditional gan",
            "cycle gan",
            "stylegan",

            # 기타 딥러닝 구조
            "deep belief network",
            "dbn",
            "restricted boltzmann machine",
            "rbm",
            "capsule network",
            "neural architecture search",
            "deep learning architecture",
        ],
    },
    "classic_ml": {
        "name_ko": "고전적 머신러닝 알고리즘",
        "keywords": [
            # SVM 계열
            "support vector machine",
            "support vector machines",
            "svm",
            "linear svm",
            "nonlinear svm",

            # 트리·앙상블
            "random forest",
            "random forests",
            "decision tree",
            "decision trees",
            "classification tree",
            "regression tree",
            "gradient boosting",
            "gradient boosted trees",
            "gradient boosting machine",
            "gbm",
            "xgboost",
            "lightgbm",
            "catboost",
            "adaboost",
            "boosting algorithm",
            "bagging algorithm",
            "bagging classifier",
            "ensemble classifier",
            "ensemble regression",

            # 최근접·확률 모델
            "k-nearest neighbor",
            "k-nearest neighbours",
            "k nearest neighbor",
            "k nearest neighbours",
            "knn",
            "naive bayes",
            "bayesian classifier",
            "bayesian network",
            "probabilistic graphical model",
            "hidden markov model",
            "hidden markov models",
            "hmm",
            "gaussian mixture model",
            "mixture of gaussians",
            "gaussian process regression",
            "gaussian process",

            # 회귀·판별분석
            "linear regression",
            "logistic regression",
            "ridge regression",
            "lasso regression",
            "elastic net",
            "discriminant analysis",
            "linear discriminant analysis",
            "lda",
            "quadratic discriminant analysis",
            "qda",

            # 군집·차원축소
            "k-means clustering",
            "kmeans clustering",
            "hierarchical clustering",
            "spectral clustering",
            "density based clustering",
            "dbscan",
            "principal component analysis",
            "pca",
            "independent component analysis",
            "ica",
            "matrix factorization",
        ],
    },
    "reinforcement_learning": {
        "name_ko": "강화학습",
        "keywords": [
            "reinforcement learning",
            "deep reinforcement learning",
            "rl",

            # MDP 계열
            "markov decision process",
            "markov decision processes",
            "mdp",
            "partially observable markov decision process",
            "pomdp",

            # 기본 알고리즘
            "q-learning",
            "q learning",
            "sarsa",
            "temporal difference learning",
            "td learning",
            "monte carlo control",
            "value iteration",
            "policy iteration",

            # 정책 경사·액터 크리틱
            "policy gradient",
            "policy gradients",
            "actor critic",
            "actor-critic",
            "deep q network",
            "deep q-learning",
            "dqn",
            "double dqn",
            "dueling dqn",
            "advantage actor critic",
            "a2c",
            "a3c",
            "proximal policy optimization",
            "ppo",
            "trust region policy optimization",
            "trpo",

            # 응용·다중 에이전트
            "multi agent reinforcement learning",
            "multi-agent reinforcement learning",
            "inverse reinforcement learning",
            "model based reinforcement learning",
            "model-free reinforcement learning",
            "reward shaping",
            "exploration exploitation",
            "exploration-exploitation",

            # 밴딧 계열
            "bandit algorithm",
            "multi armed bandit",
            "multi-armed bandit",
            "contextual bandit",
        ],
    },
    "nlp": {
        "name_ko": "자연어처리",
        "keywords": [
            "natural language processing",
            "nlp",
            "computational linguistics",

            # 언어 모델·임베딩
            "language model",
            "language models",
            "neural language model",
            "large language model",
            "large language models",
            "llm",
            "pretrained language model",
            "pre-trained language model",
            "word embedding",
            "word embeddings",
            "word2vec",
            "glove",
            "fasttext",
            "contextual embedding",
            "contextual embeddings",
            "bert",
            "roberta",
            "gpt",
            "gpt-2",
            "gpt-3",

            # 기본 NLP 과제
            "sequence labeling",
            "part of speech tagging",
            "pos tagging",
            "named entity recognition",
            "ner",
            "text classification",
            "sentiment analysis",
            "opinion mining",

            # 주제·요약
            "topic modeling",
            "latent dirichlet allocation",
            "lda topic model",
            "document clustering",
            "text clustering",

            # 번역·질문응답
            "machine translation",
            "neural machine translation",
            "question answering",
            "reading comprehension",
            "natural language inference",

            # 요약·대화
            "text summarization",
            "abstractive summarization",
            "extractive summarization",
            "dialogue system",
            "chatbot",
            "conversational agent",

            # 음성·의미 분석
            "spoken language understanding",
            "speech recognition",
            "automatic speech recognition",
            "asr",
            "speech to text",
            "text to speech",
            "semantic parsing",
            "semantic role labeling",

            # 검색·정보검색
            "document retrieval",
            "information retrieval",
            "ranking model",
        ],
    },
    "vision": {
        "name_ko": "컴퓨터 비전",
        "keywords": [
            "computer vision",

            # 이미지 처리·분석
            "image processing",
            "digital image processing",
            "image analysis",
            "visual recognition",
            "image recognition",
            "image classification",
            "image retrieval",

            # 객체·추적
            "object detection",
            "object tracking",
            "visual tracking",
            "face recognition",
            "facial recognition",
            "face detection",
            "pedestrian detection",

            # 분할 관련
            "semantic segmentation",
            "instance segmentation",
            "panoptic segmentation",
            "image segmentation",
            "video segmentation",

            # 장면·행동 인식
            "scene understanding",
            "scene classification",
            "action recognition",
            "gesture recognition",
            "human pose estimation",
            "pose estimation",

            # 기초 비전
            "optical flow",
            "stereo vision",
            "3d reconstruction",
            "structure from motion",
            "image registration",

            # 복원·향상
            "image restoration",
            "image denoising",
            "image super resolution",
            "image enhancement",

            # 비디오·감시
            "video analysis",
            "video surveillance",
            "object proposal",
            "saliency detection",
            "salient object detection",

            # 응용 분야
            "medical image analysis",
            "medical imaging",
            "radiomics",
            "remote sensing image",
            "hyperspectral image",
            "hyper-spectral image",
            "satellite image",
            "thermal image",
            "infrared image",
        ],
    },
}

# 추가적인 일반 AI 키워드 (특정 세부 분야와 직접 연결되지 않는 표현들)
AI_EXTRA_KEYWORDS = [
    "artificial intelligence",
    "intelligent system",
    "intelligent systems",
    "machine intelligence",
    "intelligent algorithm",
    "intelligent control",
    "intelligent decision support",
    "expert system",
    "expert systems",
    "knowledge based system",
    "knowledge-based system",
]

# 세부 분야 키워드 + 추가 키워드를 모두 합쳐서 중복 제거
ai_keywords_set = set(AI_EXTRA_KEYWORDS)
for area_info in AI_AREAS.values():
    ai_keywords_set.update(area_info["keywords"])

AI_KEYWORDS = sorted(ai_keywords_set)

print("AI 세부 분야 개수:", len(AI_AREAS))
print("AI 키워드 개수:", len(AI_KEYWORDS))


AI 세부 분야 개수: 6
AI 키워드 개수: 347


In [ ]:
# 셀 5: 공학 소분류별 키워드 정의 (소문자 기준으로 작성)

ENGINEERING_SUBFIELDS = {
    "arch_building": {
        "mid_ko": "건축",
        "sub_ko": "건축·설비공학",
        "keywords": [
            "building engineering",
            "building services",
            "building service engineering",
            "building system",
            "building systems",
            "hvac",
            "heating ventilation and air conditioning",
            "ventilation",
            "air conditioning",
            "thermal comfort",
            "indoor environment",
            "indoor air quality",
            "iaq",
            "building energy",
            "building performance",
            "building envelope",
            "building retrofit",
            "energy efficient building",
            "low energy building",
            "zero energy building",
            "building simulation",
            "building energy simulation",
            "building automation",
            "building management system",
            "bms",
            "indoor thermal environment",
            "thermal environment",
            "heating system",
            "cooling system",
            "district heating",
            "district cooling",
        ],
    },
    "arch_architecture": {
        "mid_ko": "건축",
        "sub_ko": "건축학",
        "keywords": [
            "architecture",
            "architectural design",
            "architectural engineering",
            "building design",
            "building layout",
            "floor plan",
            "architectural planning",
            "housing design",
            "housing planning",
            "urban housing",
            "residential building",
            "residential architecture",
            "sustainable architecture",
            "green building",
            "architectural space",
            "interior architecture",
            "building facade",
            "façade design",
            "architectural heritage",
            "historic building",
            "architectural conservation",
        ],
    },
    "arch_landscape": {
        "mid_ko": "건축",
        "sub_ko": "조경학",
        "keywords": [
            "landscape architecture",
            "landscape design",
            "urban landscape",
            "landscape planning",
            "green space",
            "green spaces",
            "green infrastructure",
            "urban green space",
            "urban green infrastructure",
            "park design",
            "urban park",
            "open space",
            "recreational space",
            "recreation area",
            "ecological landscape",
            "landscape ecology",
            "landscape restoration",
            "park planning",
            "planting design",
            "tree planting",
        ],
    },
    "civil_civil": {
        "mid_ko": "토목·도시",
        "sub_ko": "토목공학",
        "keywords": [
            "civil engineering",
            "structural engineering",
            "geotechnical engineering",
            "bridge",
            "bridges",
            "bridge engineering",
            "bridge design",
            "tunnel",
            "tunnels",
            "tunnel engineering",
            "dam",
            "dams",
            "dam engineering",
            "foundation",
            "foundation engineering",
            "soil mechanics",
            "soil stabilization",
            "slope stability",
            "landslide",
            "embankment",
            "pavement",
            "pavement design",
            "highway engineering",
            "road engineering",
            "structural analysis",
            "finite element analysis",
            "fea",
            "earthquake engineering",
            "seismic design",
            "seismic performance",
            "hydraulic engineering",
            "river engineering",
        ],
    },
    "civil_urban": {
        "mid_ko": "토목·도시",
        "sub_ko": "도시공학",
        "keywords": [
            "urban planning",
            "urban design",
            "urban engineering",
            "land use",
            "land-use",
            "land use planning",
            "zoning",
            "urban development",
            "urban redevelopment",
            "urban regeneration",
            "urban renewal",
            "smart city",
            "smart cities",
            "urban mobility",
            "urban transportation planning",
            "transportation planning",
            "urban sprawl",
            "compact city",
            "transit oriented development",
            "tod",
            "urban growth",
            "urban form",
            "urban infrastructure",
        ],
    },
    "transport_ground": {
        "mid_ko": "교통·운송",
        "sub_ko": "지상교통공학",
        "keywords": [
            "transportation engineering",
            "traffic engineering",
            "traffic flow",
            "traffic congestion",
            "traffic safety",
            "traffic signal",
            "traffic signal control",
            "signalized intersection",
            "intelligent transportation system",
            "intelligent transport system",
            "its",
            "public transport",
            "public transportation",
            "bus system",
            "bus rapid transit",
            "brt",
            "railway",
            "rail transport",
            "metro system",
            "subway system",
            "road traffic",
            "highway traffic",
            "driver behavior",
            "vehicle routing",
            "route choice",
        ],
    },
    "transport_aero": {
        "mid_ko": "교통·운송",
        "sub_ko": "항공학",
        "keywords": [
            "aerospace engineering",
            "aeronautical engineering",
            "aircraft design",
            "aircraft control",
            "flight control",
            "flight dynamics",
            "flight mechanics",
            "aerodynamics",
            "aerodynamic design",
            "propulsion system",
            "jet engine",
            "turbofan engine",
            "aeroelasticity",
            "aircraft performance",
            "aircraft stability",
            "unmanned aerial vehicle",
            "unmanned aerial vehicles",
            "uav",
            "drone",
            "air traffic management",
        ],
    },
    "transport_marine": {
        "mid_ko": "교통·운송",
        "sub_ko": "해양공학",
        "keywords": [
            "ocean engineering",
            "marine engineering",
            "naval architecture",
            "ship design",
            "ship hydrodynamics",
            "ship structure",
            "ship structures",
            "offshore structure",
            "offshore structures",
            "offshore platform",
            "offshore engineering",
            "marine structure",
            "marine structures",
            "subsea structure",
            "subsea pipeline",
            "coastal engineering",
            "coastal structure",
            "wave loading",
            "wave-structure interaction",
            "seakeeping",
        ],
    },
    "mech_mechanical": {
        "mid_ko": "기계·금속",
        "sub_ko": "기계공학",
        "keywords": [
            "mechanical engineering",
            "mechanical design",
            "mechanical system",
            "mechanical systems",
            "dynamics",
            "kinematics",
            "mechanism design",
            "mechatronics",
            "robot",
            "robots",
            "robotics",
            "robotic system",
            "robotic systems",
            "robotic arm",
            "fluid mechanics",
            "fluid dynamics",
            "heat transfer",
            "thermofluid",
            "thermodynamics",
            "thermal system",
            "vibration analysis",
            "structural vibration",
            "mechanical vibration",
            "finite element method",
            "computational fluid dynamics",
            "cfd",
        ],
    },
    "mech_metal": {
        "mid_ko": "기계·금속",
        "sub_ko": "금속공학",
        "keywords": [
            "metallurgy",
            "metallurgical engineering",
            "metallurgical",
            "metal materials",
            "metallic material",
            "metallic alloy",
            "steel",
            "stainless steel",
            "aluminum alloy",
            "titanium alloy",
            "heat treatment",
            "microstructure",
            "phase transformation",
            "microstructural evolution",
            "grain growth",
            "solidification",
            "casting",
            "forging",
            "rolling",
            "welding",
            "welded joint",
            "corrosion resistance",
        ],
    },
    "mech_automotive": {
        "mid_ko": "기계·금속",
        "sub_ko": "자동차공학",
        "keywords": [
            "automotive engineering",
            "vehicle engineering",
            "vehicle dynamics",
            "vehicle handling",
            "electric vehicle",
            "hybrid vehicle",
            "hybrid electric vehicle",
            "battery electric vehicle",
            "autonomous vehicle",
            "self-driving car",
            "advanced driver assistance system",
            "adas",
            "powertrain",
            "engine performance",
            "engine control",
            "emission control",
            "vehicle control",
            "chassis",
            "suspension",
            "braking system",
            "brake control",
            "vehicle noise vibration harshness",
            "vehicle nvh",
        ],
    },
    "elec_electrical": {
        "mid_ko": "전기·전자",
        "sub_ko": "전기공학",
        "keywords": [
            "electrical engineering",
            "power system",
            "power systems",
            "power grid",
            "smart grid",
            "distribution system",
            "distribution network",
            "transmission line",
            "transmission system",
            "electric machine",
            "electric motor",
            "induction motor",
            "synchronous motor",
            "generator",
            "transformer",
            "power electronics",
            "dc dc converter",
            "dc-ac converter",
            "inverter",
            "converter",
            "power quality",
            "fault diagnosis in power systems",
            "renewable integration",
            "microgrid",
            "distributed generation",
        ],
    },
    "elec_electronic": {
        "mid_ko": "전기·전자",
        "sub_ko": "전자공학",
        "keywords": [
            "electronics",
            "electronic engineering",
            "electronic circuit",
            "electronic circuits",
            "semiconductor device",
            "semiconductor devices",
            "integrated circuit",
            "integrated circuits",
            "ic",
            "vlsi",
            "analog circuit",
            "analog circuits",
            "digital circuit",
            "digital circuits",
            "mixed-signal",
            "mixed signal circuit",
            "signal integrity",
            "microelectronics",
            "nanoelectronics",
            "system on chip",
            "soc",
            "rf circuit",
            "microwave circuit",
            "analog to digital converter",
            "adc",
            "digital to analog converter",
            "dac",
        ],
    },
    "elec_control": {
        "mid_ko": "전기·전자",
        "sub_ko": "제어계측공학",
        "keywords": [
            "control engineering",
            "control system",
            "control systems",
            "automatic control",
            "feedback control",
            "robust control",
            "optimal control",
            "nonlinear control",
            "pid control",
            "state estimation",
            "state observer",
            "model predictive control",
            "mpc",
            "sensor",
            "sensors",
            "sensing system",
            "measurement system",
            "instrumentation",
            "sensor fusion",
            "calibration",
            "condition monitoring",
            "fault detection",
        ],
    },
    "precise_optical": {
        "mid_ko": "정밀·에너지",
        "sub_ko": "광학공학",
        "keywords": [
            "optical engineering",
            "applied optics",
            "laser system",
            "laser systems",
            "laser optics",
            "optical fiber",
            "optical fibre",
            "optical communication",
            "fiber optic communication",
            "imaging system",
            "optical sensor",
            "optical sensors",
            "photonics",
            "photonic device",
            "optical microscope",
            "optical microscopy",
            "imaging optics",
            "diffractive optics",
            "lens design",
            "optical coherence tomography",
            "oct",
        ],
    },
    "precise_energy": {
        "mid_ko": "정밀·에너지",
        "sub_ko": "에너지공학",
        "keywords": [
            "energy engineering",
            "energy system",
            "energy systems",
            "energy management",
            "renewable energy",
            "solar energy",
            "solar power",
            "photovoltaic system",
            "wind energy",
            "wind power",
            "hydro power",
            "hydropower",
            "fuel cell",
            "energy storage",
            "battery system",
            "battery energy storage",
            "thermal energy storage",
            "energy efficiency",
            "energy optimization",
            "smart energy system",
            "distributed generation",
            "demand response",
        ],
    },
    "materials_semiceramic": {
        "mid_ko": "소재·재료",
        "sub_ko": "반도체·세라믹공학",
        "keywords": [
            "semiconductor engineering",
            "semiconductor processing",
            "semiconductor process",
            "device fabrication",
            "wafer",
            "silicon wafer",
            "lithography",
            "photolithography",
            "semiconductor device",
            "mosfet",
            "cmos",
            "thin film transistor",
            "tft",
            "ceramic material",
            "ceramic materials",
            "ceramics",
            "ferroelectric",
            "ferroelectric material",
            "piezoelectric",
            "piezoelectric material",
            "dielectric material",
            "oxide semiconductor",
        ],
    },
    "materials_textile": {
        "mid_ko": "소재·재료",
        "sub_ko": "섬유공학",
        "keywords": [
            "textile engineering",
            "textile material",
            "textile materials",
            "fiber material",
            "fiber materials",
            "polymer fiber",
            "synthetic fiber",
            "natural fiber",
            "yarn",
            "weaving",
            "knitting",
            "nonwoven fabric",
            "nonwoven",
            "spinning",
            "dyeing",
            "textile dyeing",
            "finishing",
            "textile processing",
            "fabric structure",
            "fabric properties",
        ],
    },
    "materials_advanced": {
        "mid_ko": "소재·재료",
        "sub_ko": "신소재공학",
        "keywords": [
            "advanced materials",
            "novel material",
            "novel materials",
            "functional material",
            "functional materials",
            "nanomaterial",
            "nanomaterials",
            "nanostructure",
            "nanostructured material",
            "composite material",
            "composite materials",
            "nanocomposite",
            "smart material",
            "shape memory alloy",
            "sma",
            "bio material",
            "biomaterial",
            "biomaterials",
            "energy material",
            "electronic material",
        ],
    },
    "materials_material": {
        "mid_ko": "소재·재료",
        "sub_ko": "재료공학",
        "keywords": [
            "materials science",
            "materials engineering",
            "material characterization",
            "mechanical property",
            "mechanical properties",
            "tensile strength",
            "yield strength",
            "fracture toughness",
            "fracture",
            "fatigue",
            "fatigue life",
            "creep",
            "creep behavior",
            "corrosion",
            "corrosion resistance",
            "coating",
            "surface coating",
            "alloy design",
            "phase diagram",
            "microstructure",
            "microstructural analysis",
        ],
    },
    "industrial_industrial": {
        "mid_ko": "산업",
        "sub_ko": "산업공학",
        "keywords": [
            "industrial engineering",
            "systems engineering",
            "operations research",
            "production planning",
            "production scheduling",
            "scheduling",
            "supply chain",
            "supply chain management",
            "logistics",
            "logistics optimization",
            "inventory control",
            "inventory management",
            "quality control",
            "statistical quality control",
            "process optimization",
            "process improvement",
            "lean manufacturing",
            "lean production",
            "six sigma",
            "queuing theory",
            "queueing system",
            "human factors",
            "ergonomics",
            "work measurement",
        ],
    },
    "chem_chemical": {
        "mid_ko": "화공",
        "sub_ko": "화학공학",
        "keywords": [
            "chemical engineering",
            "process engineering",
            "chemical process",
            "process design",
            "process optimization",
            "process control",
            "process simulation",
            "reactor design",
            "chemical reactor",
            "reactor modeling",
            "catalysis",
            "catalytic reaction",
            "heterogeneous catalysis",
            "homogeneous catalysis",
            "distillation",
            "distillation column",
            "separation process",
            "membrane process",
            "absorption column",
            "heat exchanger",
            "polymerization",
            "polymer process",
            "reaction kinetics",
            "fluidized bed",
        ],
    },
}

# 통계 출력: 소분류 개수, 중분류 개수, 전체 키워드 개수
num_subfields = len(ENGINEERING_SUBFIELDS)
mid_categories = sorted({info["mid_ko"] for info in ENGINEERING_SUBFIELDS.values()})
num_mid = len(mid_categories)
total_keywords = sum(len(info["keywords"]) for info in ENGINEERING_SUBFIELDS.values())

print("소분류 개수:", num_subfields)
print("중분류 개수:", num_mid)
print("중분류 목록:", mid_categories)
print("전체 키워드 개수:", total_keywords)


소분류 개수: 22
중분류 개수: 9
중분류 목록: ['건축', '교통·운송', '기계·금속', '산업', '소재·재료', '전기·전자', '정밀·에너지', '토목·도시', '화공']
전체 키워드 개수: 511


In [ ]:
# 셀 6: 전처리 및 키워드 매칭 함수 정의

def combine_text_columns(df, text_columns):
    """
    여러 텍스트 컬럼(예: 제목, 초록)을 하나의 문자열로 합쳐서 소문자로 변환합니다.
    """
    text = pd.Series("", index=df.index)
    for col in text_columns:
        if col in df.columns:
            text = text + " " + df[col].fillna("").astype(str)
    return text.str.lower()

def make_or_pattern(keywords):
    """
    키워드 리스트를 정규식 OR 패턴으로 변환합니다.
    """
    escaped = [re.escape(k) for k in keywords if k]
    if not escaped:
        return None
    pattern = "(" + "|".join(escaped) + ")"
    return pattern

def make_ai_mask(text_series):
    """
    AI_KEYWORDS 또는 'ai'라는 단어를 포함하는지 여부를 반환합니다.
    텍스트는 이미 combine_text_columns에서 소문자로 변환된 것을 사용합니다.
    """
    pattern = make_or_pattern(AI_KEYWORDS)
    if pattern is None:
        base_mask = pd.Series(False, index=text_series.index)
    else:
        base_mask = text_series.str.contains(pattern, case=False, regex=True, na=False)

    # 'ai' 단어 자체도 약어로 많이 쓰이므로 별도로 포함합니다.
    ai_word_mask = text_series.str.contains(r"\bai\b", case=False, regex=True, na=False)

    return base_mask | ai_word_mask

# 공학 소분류별 정규식 패턴 생성
SUBFIELD_PATTERNS = {}
for key, info in ENGINEERING_SUBFIELDS.items():
    pattern = make_or_pattern(info["keywords"])
    SUBFIELD_PATTERNS[key] = pattern

# 인공지능 세부 분야별 정규식 패턴 생성
AI_AREA_PATTERNS = {}
for area_key, info in AI_AREAS.items():
    pattern = make_or_pattern(info["keywords"])
    AI_AREA_PATTERNS[area_key] = pattern

def detect_ai_areas(text):
    """
    한 논문의 텍스트(text_all)에서 어떤 인공지능 세부 분야 키워드들이 나타나는지
    세부 분야 키 값 리스트를 반환합니다.
    """
    text = text if isinstance(text, str) else ""
    text = text.lower()
    areas = []
    for area_key, pattern in AI_AREA_PATTERNS.items():
        if pattern and re.search(pattern, text, flags=re.IGNORECASE):
            areas.append(area_key)
    return areas

def ai_areas_to_label(areas):
    """
    세부 분야 키 리스트를 한글 라벨 문자열로 변환합니다.
    예: ["ml_general", "vision"] -> "머신러닝 일반,컴퓨터 비전"
    """
    if not areas:
        return ""
    labels = [AI_AREAS[a]["name_ko"] for a in areas if a in AI_AREAS]
    labels = sorted(set(labels))
    return ",".join(labels)

print("SUBFIELD_PATTERNS 개수:", len(SUBFIELD_PATTERNS))
print("AI_AREA_PATTERNS 개수:", len(AI_AREA_PATTERNS))


SUBFIELD_PATTERNS 개수: 22
AI_AREA_PATTERNS 개수: 6


In [ ]:
# # 셀 7: 파일 이름 파싱 + 연도/학교별 데이터 로딩

# def parse_year_and_university_from_filename(path):
#     """
#     파일 이름에서 연도와 학교 이름을 추출합니다.
#     예: works_top10000_2011_Seoul_National_University.cleaned.csv
#          -> year=2011, university='Seoul_National_University'
#     """
#     base = os.path.basename(path)
#     name = os.path.splitext(base)[0]  # .csv 제거 (있다면)

#     # .cleaned로 끝나면 제거
#     if name.endswith(".cleaned"):
#         name = name[: -len(".cleaned")]

#     prefix = "works_top10000_"
#     if name.startswith(prefix):
#         name = name[len(prefix):]

#     parts = name.split("_", 1)
#     if len(parts) != 2:
#         return None, None
#     year_str, uni_str = parts
#     try:
#         year = int(year_str)
#     except ValueError:
#         return None, None
#     return year, uni_str

# def load_all_university_data(base_dir, year_folder_map):
#     """
#     연도별 폴더에서 학교별 파일을 모두 읽어 하나의 DataFrame으로 합칩니다.
#     ALL 파일은 제외하고, 학교별 파일만 사용합니다.
#     """
#     dfs = []

#     for year, folder_name in year_folder_map.items():
#         folder_path = os.path.join(base_dir, folder_name)
#         if not os.path.isdir(folder_path):
#             print(f"[경고] {year}년 폴더가 존재하지 않습니다: {folder_path}")
#             continue

#         pattern = os.path.join(folder_path, f"works_top10000_{year}_*.cleaned*")
#         files = glob.glob(pattern)

#         if not files:
#             print(f"[경고] {year}년에서 매칭되는 파일이 없습니다: {pattern}")
#             continue

#         print(f"{year}년: 파일 {len(files)}개 발견")

#         for path in files:
#             y, uni_str = parse_year_and_university_from_filename(path)
#             if y is None or uni_str is None:
#                 print("  [무시] 파일 이름에서 연도/학교를 파싱하지 못했습니다:", os.path.basename(path))
#                 continue

#             # ALL 파일은 제외
#             if uni_str.upper() == "ALL":
#                 continue

#             try:
#                 df = pd.read_csv(path)
#             except Exception as e:
#                 print("  [무시] CSV를 읽는 중 오류:", os.path.basename(path), "오류:", e)
#                 continue

#             df["year_from_filename"] = y
#             df["university_from_filename"] = uni_str.replace("_", " ")
#             dfs.append(df)

#     if dfs:
#         df_all = pd.concat(dfs, ignore_index=True)
#     else:
#         df_all = pd.DataFrame()

#     return df_all

# df_all = load_all_university_data(BASE_DIR, YEAR_FOLDER_MAP)
# print("전체 행 수:", len(df_all))

# if not df_all.empty:
#     print(df_all[["year_from_filename", "university_from_filename"]].drop_duplicates().head())


In [ ]:
# 셀 7: 파일 이름 파싱 + 연도/학교별 데이터 로딩 (체크포인트 적용 버전)

def parse_year_and_university_from_filename(path):
    base = os.path.basename(path)
    name = os.path.splitext(base)[0]
    if name.endswith(".cleaned"):
        name = name[: -len(".cleaned")]
    prefix = "works_top10000_"
    if name.startswith(prefix):
        name = name[len(prefix):]
    parts = name.split("_", 1)
    if len(parts) != 2:
        return None, None
    year_str, uni_str = parts
    try:
        year = int(year_str)
    except ValueError:
        return None, None
    return year, uni_str

def load_all_university_data(base_dir, year_folder_map):
    dfs = []
    for year, folder_name in year_folder_map.items():
        folder_path = os.path.join(base_dir, folder_name)
        if not os.path.isdir(folder_path):
            print(f"[경고] {year}년 폴더가 존재하지 않습니다: {folder_path}")
            continue

        pattern = os.path.join(folder_path, f"works_top10000_{year}_*.cleaned*")
        files = glob.glob(pattern)
        if not files:
            print(f"[경고] {year}년에서 매칭되는 파일이 없습니다: {pattern}")
            continue

        print(f"{year}년: 파일 {len(files)}개 발견")
        for path in files:
            y, uni_str = parse_year_and_university_from_filename(path)
            if y is None or uni_str is None:
                print("  [무시] 파싱 실패:", os.path.basename(path))
                continue
            if uni_str.upper() == "ALL":
                continue

            try:
                df = pd.read_csv(path)
            except Exception as e:
                print("  [무시] CSV 읽기 오류:", os.path.basename(path), "오류:", e)
                continue

            df["year_from_filename"] = y
            df["university_from_filename"] = uni_str.replace("_", " ")
            dfs.append(df)

    if dfs:
        df_all = pd.concat(dfs, ignore_index=True)
    else:
        df_all = pd.DataFrame()
    return df_all

# 체크포인트 경로
df_all_ckpt_path = os.path.join(CHECKPOINT_DIR, "df_all.csv")

if os.path.exists(df_all_ckpt_path):
    print("체크포인트에서 df_all을 불러옵니다:", df_all_ckpt_path)
    df_all = pd.read_csv(df_all_ckpt_path)
else:
    print("df_all 체크포인트가 없어 새로 로딩합니다.")
    df_all = load_all_university_data(BASE_DIR, YEAR_FOLDER_MAP)
    print("df_all 전체 행 수:", len(df_all))
    if not df_all.empty:
        df_all.to_csv(df_all_ckpt_path, index=False, encoding="utf-8-sig")
        print("df_all을 체크포인트로 저장했습니다:", df_all_ckpt_path)


df_all 체크포인트가 없어 새로 로딩합니다.
2011년: 파일 24개 발견
2012년: 파일 24개 발견
2013년: 파일 24개 발견
2014년: 파일 24개 발견
2015년: 파일 24개 발견
2016년: 파일 24개 발견
2017년: 파일 24개 발견
2018년: 파일 24개 발견
2019년: 파일 24개 발견
2020년: 파일 24개 발견
2021년: 파일 24개 발견
2022년: 파일 24개 발견
2023년: 파일 24개 발견
2024년: 파일 24개 발견
2025년: 파일 24개 발견
df_all 전체 행 수: 819822
df_all을 체크포인트로 저장했습니다: /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/checkpoints/df_all.csv


In [ ]:
# # 셀 8: 텍스트 결합 및 AI 관련 논문 필터링

# if df_all.empty:
#     print("df_all이 비어 있습니다. YEAR_FOLDER_MAP과 파일 경로를 먼저 확인해 주세요.")
# else:
#     # 제목 + 초록 등을 하나의 텍스트로 합치기
#     df_all = df_all.copy()
#     df_all["text_all"] = combine_text_columns(df_all, TEXT_COLUMNS)

#     print("text_all 예시:")
#     print(df_all["text_all"].head())

#     # AI 관련 키워드 포함 여부
#     df_all["is_ai"] = make_ai_mask(df_all["text_all"])
#     df_ai = df_all[df_all["is_ai"]].copy()

#     print("AI 관련 논문 수:", len(df_ai))
#     print(df_ai[["year_from_filename", "university_from_filename"]].head())


In [ ]:
# 셀 8: 텍스트 결합 및 AI 관련 논문 필터링 (체크포인트 적용)

df_ai_ckpt_path = os.path.join(CHECKPOINT_DIR, "df_ai.csv")

if os.path.exists(df_ai_ckpt_path):
    print("체크포인트에서 df_ai를 불러옵니다:", df_ai_ckpt_path)
    df_ai = pd.read_csv(df_ai_ckpt_path)
    # text_all이 필요하면 같이 저장해두거나, 여기서 다시 만들 수도 있습니다.
    if "text_all" not in df_ai.columns:
        print("text_all이 없어 재생성합니다.")
        df_ai["text_all"] = combine_text_columns(df_ai, TEXT_COLUMNS)
else:
    if df_all.empty:
        print("df_all이 비어 있습니다. 셀 7을 먼저 실행해 주세요.")
    else:
        df_all = df_all.copy()
        df_all["text_all"] = combine_text_columns(df_all, TEXT_COLUMNS)
        df_all["is_ai"] = make_ai_mask(df_all["text_all"])
        df_ai = df_all[df_all["is_ai"]].copy()
        print("AI 관련 논문 수:", len(df_ai))
        df_ai.to_csv(df_ai_ckpt_path, index=False, encoding="utf-8-sig")
        print("df_ai를 체크포인트로 저장했습니다:", df_ai_ckpt_path)


/tmp/ipython-input-2926036018.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  base_mask = text_series.str.contains(pattern, case=False, regex=True, na=False)


AI 관련 논문 수: 634135
df_ai를 체크포인트로 저장했습니다: /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/checkpoints/df_ai.csv


In [11]:
# 셀 9: 공학 소분류 키워드 매칭 및 AI+공학 논문 추출, 인공지능 세부 분야 라벨링
#       + df_ai_eng에 대한 체크포인트 저장/불러오기

df_ai_eng_ckpt_path = os.path.join(CHECKPOINT_DIR, "df_ai_eng.csv")

# 1) 이미 체크포인트가 있으면 먼저 불러오기
if os.path.exists(df_ai_eng_ckpt_path):
    print("체크포인트에서 df_ai_eng를 불러옵니다:", df_ai_eng_ckpt_path)
    df_ai_eng = pd.read_csv(df_ai_eng_ckpt_path)

    # 열이 제대로 있는지 간단히 확인
    expected_cols = {"year_from_filename", "university_from_filename",
                     "subfield_mid_ko", "subfield_sub_ko",
                     "text_all", "ai_areas", "ai_areas_ko"}
    missing = expected_cols - set(df_ai_eng.columns)
    if missing:
        print("[주의] df_ai_eng에 예상되는 컬럼이 없습니다:", missing)
    else:
        print("df_ai_eng shape:", df_ai_eng.shape)
        print(
            df_ai_eng[
                [
                    "year_from_filename",
                    "university_from_filename",
                    "subfield_mid_ko",
                    "subfield_sub_ko",
                    "ai_areas_ko",
                ]
            ].head()
        )

# 2) 체크포인트가 없으면 새로 계산
else:
    if df_all.empty:
        print("df_all이 비어 있습니다. 먼저 데이터 로딩과 AI 필터링을 확인해 주세요.")
    elif "is_ai" not in df_all.columns:
        print("df_all에 is_ai가 없습니다. 셀 8을 먼저 실행해 주세요.")
    else:
        df_ai = df_all[df_all["is_ai"]].copy()

        ai_eng_dfs = []

        for sub_key, info in ENGINEERING_SUBFIELDS.items():
            pattern = SUBFIELD_PATTERNS.get(sub_key)
            if not pattern:
                continue

            mask_sub = df_ai["text_all"].str.contains(
                pattern, case=False, regex=True, na=False
            )
            sub_df = df_ai[mask_sub].copy()

            if sub_df.empty:
                continue

            sub_df["subfield_key"] = sub_key
            sub_df["subfield_mid_ko"] = info["mid_ko"]
            sub_df["subfield_sub_ko"] = info["sub_ko"]

            ai_eng_dfs.append(sub_df)

        if ai_eng_dfs:
            df_ai_eng = pd.concat(ai_eng_dfs, ignore_index=True)

            # 인공지능 세부 분야 라벨링
            df_ai_eng["ai_areas"] = df_ai_eng["text_all"].apply(detect_ai_areas)
            df_ai_eng["ai_areas_ko"] = df_ai_eng["ai_areas"].apply(ai_areas_to_label)

            print("AI + 공학 소분류에 동시에 해당하는 논문 수:", len(df_ai_eng))
            print(
                df_ai_eng[
                    [
                        "year_from_filename",
                        "university_from_filename",
                        "subfield_mid_ko",
                        "subfield_sub_ko",
                        "ai_areas_ko",
                    ]
                ].head()
            )

            # 체크포인트로 저장
            df_ai_eng.to_csv(df_ai_eng_ckpt_path, index=False, encoding="utf-8-sig")
            print("df_ai_eng를 체크포인트로 저장했습니다:", df_ai_eng_ckpt_path)
        else:
            df_ai_eng = pd.DataFrame()
            print("현재 키워드 설정으로 매칭되는 AI+공학 논문이 없습니다. 키워드 범위를 조정해 보셔도 됩니다.")


/tmp/ipython-input-4220756058.py:48: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_sub = df_ai["text_all"].str.contains(
/tmp/ipython-input-4220756058.py:48: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_sub = df_ai["text_all"].str.contains(
/tmp/ipython-input-4220756058.py:48: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_sub = df_ai["text_all"].str.contains(
/tmp/ipython-input-4220756058.py:48: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_sub = df_ai["text_all"].str.contains(
/tmp/ipython-input-4220756058.py:48: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get

AI + 공학 소분류에 동시에 해당하는 논문 수: 1036399
   year_from_filename     university_from_filename subfield_mid_ko  \
0                2011  Chonnam National University              건축   
1                2011  Chonnam National University              건축   
2                2011  Chonnam National University              건축   
3                2011  Chonnam National University              건축   
4                2011  Chonnam National University              건축   

  subfield_sub_ko    ai_areas_ko  
0         건축·설비공학  고전적 머신러닝 알고리즘  
1         건축·설비공학   신경망 및 딥러닝 구조  
2         건축·설비공학   신경망 및 딥러닝 구조  
3         건축·설비공학  고전적 머신러닝 알고리즘  
4         건축·설비공학          자연어처리  
df_ai_eng를 체크포인트로 저장했습니다: /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/checkpoints/df_ai_eng.csv


In [12]:
# 셀 10: 학교별·공학 소분류별 AI 논문 수 집계 예시

if df_ai_eng.empty:
    print("df_ai_eng가 비어 있습니다. 셀 9까지의 결과를 먼저 확인해 주세요.")
else:
    # 학교별 + 공학 소분류별 논문 수
    grouped_uni_sub = (
        df_ai_eng
        .groupby(["university_from_filename", "subfield_mid_ko", "subfield_sub_ko"])
        .size()
        .reset_index(name="paper_count")
        .sort_values("paper_count", ascending=False)
    )

    print("학교별, 공학 소분류별 AI 접목 논문 수 상위 20개")
    print(grouped_uni_sub.head(20))

    # 연도별 + 공학 중분류별 추세
    grouped_year_mid = (
        df_ai_eng
        .groupby(["year_from_filename", "subfield_mid_ko"])
        .size()
        .reset_index(name="paper_count")
        .sort_values(["year_from_filename", "paper_count"], ascending=[True, False])
    )

    print("\n연도별, 공학 중분류별 AI 접목 논문 수 예시 (상위 20행)")
    print(grouped_year_mid.head(20))


학교별, 공학 소분류별 AI 접목 논문 수 상위 20개
                              university_from_filename subfield_mid_ko  \
398                          Seoul National University           전기·전자   
486                                  Yonsei University           전기·전자   
442                            Sungkyunkwan University           전기·전자   
274  Korea Advanced Institute of Science and Techno...           전기·전자   
332                      Kyungpook National University           전기·전자   
376                          Pusan National University           전기·전자   
81                        Chungnam National University           전기·전자   
37                                Chung-Ang University           전기·전자   
354        Pohang University of Science and Technology           전기·전자   
230                        Jeonbuk National University           전기·전자   
15                         Chonnam National University           전기·전자   
167                     Gyeongsang National University           전기·전자   
252    

In [13]:
# 셀 11: 분석 결과를 CSV로 저장

if df_ai_eng.empty:
    print("df_ai_eng가 비어 있습니다. 셀 9까지의 결과를 먼저 확인해 주세요.")
else:
    output_dir = os.path.join(BASE_DIR, "analysis_results")
    os.makedirs(output_dir, exist_ok=True)

    # 학교별, 공학 소분류별 AI 접목 논문 수
    grouped_uni_sub_path = os.path.join(output_dir, "ai_eng_by_university_subfield.csv")
    grouped_uni_sub.to_csv(grouped_uni_sub_path, index=False, encoding="utf-8-sig")

    # 연도별, 공학 중분류별 AI 접목 논문 수
    grouped_year_mid_path = os.path.join(output_dir, "ai_eng_by_year_midfield.csv")
    grouped_year_mid.to_csv(grouped_year_mid_path, index=False, encoding="utf-8-sig")

    # AI + 공학 + 인공지능 세부 분야 전체 데이터
    full_ai_eng_path = os.path.join(output_dir, "ai_eng_papers_full_with_ai_areas.csv")
    df_ai_eng.to_csv(full_ai_eng_path, index=False, encoding="utf-8-sig")

    print("저장 완료:")
    print(" -", grouped_uni_sub_path)
    print(" -", grouped_year_mid_path)
    print(" -", full_ai_eng_path)


저장 완료:
 - /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/analysis_results/ai_eng_by_university_subfield.csv
 - /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/analysis_results/ai_eng_by_year_midfield.csv
 - /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/analysis_results/ai_eng_papers_full_with_ai_areas.csv


In [14]:
import os
import pandas as pd

output_dir = os.path.join(BASE_DIR, "analysis_results")
grouped_uni_sub_path = os.path.join(output_dir, "ai_eng_by_university_subfield.csv")
grouped_year_mid_path = os.path.join(output_dir, "ai_eng_by_year_midfield.csv")
full_ai_eng_path = os.path.join(output_dir, "ai_eng_papers_full_with_ai_areas.csv")

print("BASE_DIR:", BASE_DIR)
print("output_dir:", output_dir)
print("파일 경로 확인:")
print(" -", grouped_uni_sub_path, os.path.exists(grouped_uni_sub_path))
print(" -", grouped_year_mid_path, os.path.exists(grouped_year_mid_path))
print(" -", full_ai_eng_path, os.path.exists(full_ai_eng_path))

# 파일 크기와 행 수도 같이 확인
if os.path.exists(full_ai_eng_path):
    print("\nfull_ai_eng 파일 사이즈(byte):", os.path.getsize(full_ai_eng_path))
    df_check = pd.read_csv(full_ai_eng_path)
    print("full_ai_eng 행 수, 열 수:", df_check.shape)
    print("열 이름:", list(df_check.columns))

print("df_ai_eng shape:", df_ai_eng.shape)


BASE_DIR: /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트
output_dir: /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/analysis_results
파일 경로 확인:
 - /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/analysis_results/ai_eng_by_university_subfield.csv True
 - /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/analysis_results/ai_eng_by_year_midfield.csv True
 - /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/analysis_results/ai_eng_papers_full_with_ai_areas.csv True

full_ai_eng 파일 사이즈(byte): 5457952046
full_ai_eng 행 수, 열 수: (1036399, 43)
열 이름: ['openalex_id', 'title', 'publication_year', 'publication_date', 'type', 'host_venue', 'cited_by_count', 'is_oa', 'oa_status', 'language', 'doi', 'abstract', 'authors_count', 'author_ids', 'author_names', 'author_positions', 'author_orcids', 'author_affiliations', 'query_name', 'display_name', 'institution_id', '_garbled_score', 'title_clean', 'host_venue_clean', 'language_clean', 'abstract_clean', 'auth